# Tugas Besar IF4074 - Bembelajaran Mesin Lanjut 

## Anggota Kelompok

1. Ilham Pratama (13520041)
2. Jeremy S.O.N Simbolon (13520042)

## Kode Program

### Import semua librry yang diperlukan dalam pengerjaan tugas ini

In [291]:
import os
import random

import cv2
import numpy as np
import numpy.typing as npt

### Kelas Utils, Kelas ini digunakan untuk me-<i>load</i> dataset dan dan mengubah gambar menjadi array

In [292]:
class Utils:
    @staticmethod
    def load_dataset(dataset_path: str) -> tuple[npt.NDArray, npt.NDArray, dict]:
        folder_list = sorted(os.listdir(dataset_path))
        folder_path = []
        class_label = np.array([], dtype=np.int16)
        class_dictionary = {}
        for i, folder_name in enumerate(folder_list):
            class_folder_path = os.path.join(dataset_path, folder_name)
            list_image_name = sorted(os.listdir(class_folder_path))
            temp_folder_path = [
                os.path.join(class_folder_path, image_name)
                for image_name in list_image_name
            ]

            folder_path += temp_folder_path
            temp_class_label = np.full(len(list_image_name), i, dtype=np.int16)
            class_label = np.concatenate((class_label, temp_class_label), axis=0)
            class_dictionary[str(i)] = folder_name

        return np.asarray(folder_path), class_label, class_dictionary

    @staticmethod
    def convert_image_to_matrix(folder_path: str) -> npt.NDArray:
        list_of_image_matrix = []
        size = (256, 256)

        for file_img in folder_path:
            image = cv2.imread(file_img, 1)
            image_matrix = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image_matrix = cv2.resize(image_matrix, size)
            list_of_image_matrix.append(image_matrix)
            
        return np.array(list_of_image_matrix)


# For testing purposes
# if __name__ == "__main__":
#     folder_path, class_label, class_dictionary = Utils.load_dataset("./dataset")
#     print(Utils.convert_image_to_matrix(folder_path))

### Class Model, kelas ini berisi semua layers, fungsi, dan prosedure yang digunakan untuk proses CNN

In [293]:
class Model:
    def __init__(self) -> None:
        self._layers = []
        self.result = []

    class Layer:
        def __init__(self, name):
            self.name = name

        def feed_forward(self):
            print(f"Performing feed forward on {self.name} layer...")
            print()

    class ConvolutionLayer(Layer):
        def __init__(
            self,
            filter_size: list[tuple[int]],
            padding_size: int,
            stride_size: list[int],
        ) -> None:
            super().__init__("convolution")
            self._filter_weight = np.array(
                [
                    [
                        [random.random() for _ in range(filter_size[i][1])]
                        for _ in range(filter_size[i][0])
                    ]
                    for i in range(len(filter_size))
                ]
            )
            self._padding_size = padding_size
            self._stride_size = stride_size

        @staticmethod
        def _pad_weights(
            weights: npt.NDArray[npt.NDArray[npt.NDArray[float]]],
            padding_size: int,
        ) -> npt.NDArray[npt.NDArray[npt.NDArray[float]]]:
            padded_weights = []

            for i in range(len(weights)):
                weight_height = len(weights[i])
                weight_width = len(weights[i][0])
                padded_height = weight_height + 2 * padding_size
                padded_width = weight_width + 2 * padding_size

                new_weight = [
                    [
                        weights[i][j - padding_size][k - padding_size]
                        if padding_size <= j < weight_height + padding_size
                        or padding_size <= k < weight_width + padding_size
                        else 0.0
                        for k in range(padded_width)
                    ]
                    for j in range(padded_height)
                ]

                padded_weights.append(new_weight)

            return np.array(padded_weights)

        def convolute(
            self,
            weights: npt.NDArray[npt.NDArray[npt.NDArray[float]]],
        ) -> npt.NDArray[npt.NDArray[npt.NDArray[float]]]:
            feature_maps = []
            print("Randomly initialised filter weight: ")
            print(self._filter_weight)
            print()
            weights = self._pad_weights(weights, self._padding_size)
            for i in range(len(weights)):
                feature_map = []
                for j in range(
                    0,
                    len(weights[i]) - len(self._filter_weight[i]) + 1,
                    self._stride_size[i],
                ):
                    feature_row = []
                    for k in range(
                        0,
                        len(weights[i][j]) - len(self._filter_weight[i][0]) + 1,
                        self._stride_size[i],
                    ):
                        field = weights[
                            i,
                            j : j + len(self._filter_weight[i]),
                            k : k + len(self._filter_weight[i][0]),
                        ]
                        feature = field * self._filter_weight[i]
                        feature_row.append(np.sum(feature))
                    feature_map.append(feature_row)
                feature_maps.append(feature_map)
            return np.array(feature_maps)

        def feed_forward(
            self, weights: npt.NDArray[npt.NDArray[npt.NDArray[float]]]
        ) -> npt.NDArray[npt.NDArray[npt.NDArray[float]]]:
            super().feed_forward()
            result = self.convolute(weights)
            print("Convolution result: ")
            print(result)
            print()
            return result

    class DetectorLayer(Layer):
        def __init__(self):
            super().__init__("detector")

        @staticmethod
        def detect(feature: npt.NDArray) -> npt.NDArray:
            return np.maximum(feature, 0)

        def feed_forward(self, feature: npt.NDArray) -> npt.NDArray:
            super().feed_forward()
            result = self.detect(feature)
            print("Detector result: ")
            print(result)
            print()
            return result

    class PoolingLayer(Layer):
        def __init__(
            self, filter_size: int, stride_size: int, mode: str = "max"
        ) -> None:
            super().__init__("pooling")
            self.filter_size = filter_size
            self.stride_size = stride_size
            self.mode = mode

        def average(
            self, input_matrix: npt.NDArray, d: int, h: int, w: int
        ) -> npt.NDArray:
            h_start = h * self.stride_size
            w_start = w * self.stride_size
            h_end = h_start + self.filter_size
            w_end = w_start + self.filter_size
            return np.average(input_matrix[d, h_start:h_end, w_start:w_end])

        def max(self, input_matrix: npt.NDArray, d: int, h: int, w: int) -> npt.NDArray:
            h_start = h * self.stride_size
            w_start = w * self.stride_size
            h_end = h_start + self.filter_size
            w_end = w_start + self.filter_size
            return np.max(input_matrix[d, h_start:h_end, w_start:w_end])

        def pool(self, input_matrix: npt.NDArray) -> npt.NDArray:
            depth, height, width = input_matrix.shape
            filter_height = (height - self.filter_size) // self.stride_size + 1
            filter_width = (width - self.filter_size) // self.stride_size + 1
            pooled = np.zeros([depth, filter_height, filter_width], dtype=np.double)
            for d in range(0, depth):
                for h in range(0, filter_height):
                    for w in range(0, filter_width):
                        if self.mode == "average":
                            pooled[d, h, w] = self.average(input_matrix, d, h, w)
                        elif self.mode == "max":
                            pooled[d, h, w] = self.max(input_matrix, d, h, w)
            return pooled

        def feed_forward(self, input_matrix: npt.NDArray) -> npt.NDArray:
            super().feed_forward()
            result = self.pool(input_matrix)
            print("Pooling result: ")
            print(result)
            print()
            return result

    class DenseLayer(Layer):
        def __init__(self, unit_count: int, activation: str = "sigmoid") -> None:
            super().__init__("dense")
            self.unit_count = unit_count
            self.activation = activation
            self.bias = np.zeros(unit_count)
            self.weight = np.random.randn(unit_count)

        def dense(self, input_matrix: npt.NDArray) -> float:
            result = np.zeros(self.unit_count)

            for i in range(self.unit_count):
                input_weight = np.sum(self.weight[i] * input_matrix)
                result[i] = input_weight + self.bias[i]

            if self.activation == "sigmoid":
                return 1 / (1 + np.exp(-result))
            elif self.activation == "relu":
                return np.maximum(result, 0)

        def feed_forward(self, input_matrix: npt.NDArray) -> float:
            super().feed_forward()
            result = self.dense(input_matrix)
            print("Dense result: ")
            print(result)
            print()
            return result

    class FlattenLayer(Layer):
        def __init__(self):
            super().__init__("flatten")

        @staticmethod
        def flatten(input_matrix: npt.NDArray) -> npt.NDArray:
            return input_matrix.flatten()

        def feed_forward(self, input_matrix: npt.NDArray) -> npt.NDArray:
            super().feed_forward()
            result = self.flatten(input_matrix)
            print("Flatten result: ")
            print(result)
            print()
            return result

    def add_layer(self, name: str, **kwargs):
        match name:
            case "convolution":
                self._layers.append(self.ConvolutionLayer(**kwargs))
            case "detector":
                self._layers.append(self.DetectorLayer())
            case "pooling":
                self._layers.append(self.PoolingLayer(**kwargs))
            case "dense":
                self._layers.append(self.DenseLayer(**kwargs))
            case "flatten":
                self._layers.append(self.FlattenLayer())

    def feed_forward(self, tensor: npt.NDArray) -> None:
        for layer in self._layers:
            tensor = layer.feed_forward(tensor)
        print("Feedforward result: ")
        print(tensor)
        self.result = tensor

    def back_propagate(self) -> None:
        pass

### Kode ini merupakan main program yang digunakan pada tugas ini

In [294]:
input_tensor = np.array(
    [
        [
            [1, 2, 3, 4, 5, 6],
            [7, 8, 9, 10, 11, 12],
            [13, 14, 15, 16, 17, 18],
            [19, 20, 21, 22, 23, 24],
            [25, 26, 27, 28, 29, 30],
            [31, 32, 33, 34, 35, 36],
        ]
    ]
)
#load dataset
folder_path, class_label, class_dictionary = Utils.load_dataset("./dataset")
image_matrix = Utils.convert_image_to_matrix(folder_path)
image_number = 0
image_matrix_to_test = [image_matrix[image_number]]

model = Model()
model.add_layer("convolution", filter_size=[(3, 3)], padding_size=0, stride_size=[1])
model.add_layer("detector")
model.add_layer("pooling", filter_size=2, stride_size=1, mode="average")
model.add_layer("dense", unit_count=1, activation="sigmoid")
model.add_layer("flatten")
model.feed_forward(image_matrix_to_test)

# belum tau benar atau salahnya
# dest_class = folder_path[image_number].split("\\")
# dest_class = dest_class[-2]
# if dest_class == 'bears':
#     class_dictionary = {'0': 'pandas', '1': 'bears'}
# else:
#     class_dictionary = {'0': 'bears', '1': 'pandas'}

# print("picture : ", folder_path[image_number])
# # Jika mendekati 1 maka hasil prediksi hampir benar
# result = 0 if model.result[0] < 0.5 else 1
# print("prediction : ", class_dictionary[str(int(result))])

Performing feed forward on convolution layer...

Randomly initialised filter weight: 
[[[0.40980417 0.42932282 0.02301019]
  [0.38184355 0.71643347 0.56917264]
  [0.0953937  0.78254084 0.34766899]]]

Convolution result: 
[[[667.16115308 658.50597283 655.61613209 ... 629.38064987 625.57686601
   628.57364597]
  [657.52379749 651.83215989 652.91901318 ... 631.54677035 627.7429865
   630.71675626]
  [653.84374193 650.271887   653.34216468 ... 631.96706141 628.16327755
   630.78937832]
  ...
  [278.29100207 218.42723524 149.17760919 ...  30.94743751  77.42794817
   167.95259033]
  [201.58156754 120.33195868 110.97319285 ...  47.43233879  93.4727082
   183.29793075]
  [157.8243867   86.9088789   89.89269475 ...  85.12772251 114.76629829
   188.68871392]]]

Performing feed forward on detector layer...

Detector result: 
[[[667.16115308 658.50597283 655.61613209 ... 629.38064987 625.57686601
   628.57364597]
  [657.52379749 651.83215989 652.91901318 ... 631.54677035 627.7429865
   630.7167562

C:\Users\pilha\AppData\Local\Temp\ipykernel_17736\1715336039.py:184: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-result))
